In [16]:
import pandas as pd

In [17]:
nyc_complaint_df = pd.read_csv('data/NYPD_Complaint_Data_Historic_20240415.csv')

In [18]:
# go to https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i/about_data for information about the columns
print(len(nyc_complaint_df))
print(nyc_complaint_df.columns)

856507
Index(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'ADDR_PCT_CD', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD',
       'PD_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'BORO_NM',
       'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'JURIS_DESC', 'JURISDICTION_CODE',
       'PARKS_NM', 'HADEVELOPT', 'HOUSING_PSA', 'X_COORD_CD', 'Y_COORD_CD',
       'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT',
       'Latitude', 'Longitude', 'Lat_Lon', 'PATROL_BORO', 'STATION_NAME',
       'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Zip Codes',
       'Community Districts', 'Borough Boundaries', 'City Council Districts',
       'Police Precincts'],
      dtype='object')


In [19]:
EPA_smart_location_df = pd.read_csv('data/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv')

In [20]:
print(len(EPA_smart_location_df))
EPA_smart_location_df.columns

220740


Index(['OBJECTID', 'GEOID10', 'GEOID20', 'STATEFP', 'COUNTYFP', 'TRACTCE',
       'BLKGRPCE', 'CSA', 'CSA_Name', 'CBSA',
       ...
       'D5DRI', 'D5DE', 'D5DEI', 'D2A_Ranked', 'D2B_Ranked', 'D3B_Ranked',
       'D4A_Ranked', 'NatWalkInd', 'Shape_Length', 'Shape_Area'],
      dtype='object', length=117)

In [21]:
# TODO: find how to filter for NYC metro area, and also group by neighborhood and group in a way that matches with sentiment data
# filter by state/county fips

In [22]:
# extracting the county FIPS code for each of the nyc counties
ny_county_df = pd.read_csv('data/NY_Municipalities_and_County_FIPS_codes_20240415.csv')
nyc_counties = ['Bronx', 'Kings', 'New York', 'Queens', 'Richmond']
# filter for nyc counties
nyc_counties = ny_county_df[ny_county_df['County Name'].isin(nyc_counties)]
print(nyc_counties.head())
county_fips_lst = nyc_counties['County FIPS'].to_list()
# remove state part from county fips code (keep last 3 numbers)
county_fips_lst = [fips % 1000 for fips in county_fips_lst]
print(county_fips_lst)

   County Name      City Name Town Name Village Name   Municipality  \
6        Bronx          Bronx       NaN          NaN          Bronx   
7        Kings       Brooklyn       NaN          NaN       Brooklyn   
30    New York      Manhattan       NaN          NaN      Manhattan   
49      Queens         Queens       NaN          NaN         Queens   
58    Richmond  Staten Island       NaN          NaN  Staten Island   

    County FIPS  
6         36005  
7         36047  
30        36061  
49        36081  
58        36085  
[5, 47, 61, 81, 85]


In [25]:
# Now filter EPA data for only NYC counties using county FIPS
# first filter for NY state (36)
EPA_NYC_df = EPA_smart_location_df[EPA_smart_location_df['STATEFP'] == 36]
# then filter by 5 NYC counties
EPA_NYC_df = EPA_NYC_df[EPA_NYC_df['COUNTYFP'].isin(county_fips_lst)]
print(EPA_NYC_df.head(), EPA_NYC_df.shape)

        OBJECTID       GEOID10       GEOID20  STATEFP  COUNTYFP  TRACTCE  \
148836    148837  3.608100e+11  3.608100e+11       36        81    25900   
148837    148838  3.608100e+11  3.608100e+11       36        81    16600   
148838    148839  3.608110e+11  3.608110e+11       36        81   122702   
148839    148840  3.608100e+11  3.608100e+11       36        81    28300   
148840    148841  3.608110e+11  3.608110e+11       36        81   148300   

        BLKGRPCE    CSA                      CSA_Name     CBSA  ...     D5DRI  \
148836         2  408.0  New York-Newark, NY-NJ-CT-PA  35620.0  ...  0.506986   
148837         2  408.0  New York-Newark, NY-NJ-CT-PA  35620.0  ...  0.029180   
148838         1  408.0  New York-Newark, NY-NJ-CT-PA  35620.0  ...  0.154780   
148839         5  408.0  New York-Newark, NY-NJ-CT-PA  35620.0  ...  0.535719   
148840         4  408.0  New York-Newark, NY-NJ-CT-PA  35620.0  ...  0.082963   

            D5DE     D5DEI  D2A_Ranked  D2B_Ranked  D3B_

In [27]:
print(EPA_NYC_df['COUNTYFP'].nunique())
print(EPA_NYC_df['TRACTCE'].nunique())

5
1335


It appears that there's only 1335 unique census tracks in this data for NYC after filtering, even though there's 2327 IRL allegedly - THIS IS FALSE - We will show below, but 'TRACTCE' is only unique within each borough

In [28]:
EPA_NYC_df.to_csv('github_data/EPA_NYC_out.csv', index=False)

In [29]:
NYC_tracts_df = pd.read_csv('data/2020_Census_Tracts_-_Tabular_20240415.csv')

In [35]:
print(NYC_tracts_df['CT2020'].nunique())
print(NYC_tracts_df['BoroCT2020'].nunique())
print(len(NYC_tracts_df))

1530
2325
2325


In [39]:
unique_groups = EPA_NYC_df.groupby(['TRACTCE', 'BLKGRPCE']).ngroups
print("Number of unique combinations:", unique_groups)
print(EPA_NYC_df['GEOID10'].nunique())
print(EPA_NYC_df['GEOID20'].nunique())
unique_groups = EPA_NYC_df.groupby(['TRACTCE', 'COUNTYFP']).ngroups
print("Number of unique combinations:", unique_groups)

Number of unique combinations: 4474
13
13
Number of unique combinations: 2167


In [33]:
print(EPA_NYC_df['TRACTCE'])
print(NYC_tracts_df['BoroCT2020'])

148836     25900
148837     16600
148838    122702
148839     28300
148840    148300
           ...  
164267     22401
164273     38900
164274      7100
164275     21100
164276     19700
Name: TRACTCE, Length: 6493, dtype: int64
0       1000100
1       1000201
2       1000600
3       1001401
4       1001402
         ...   
2320    2033400
2321    5022602
2322    5024800
2323    4003400
2324    3118600
Name: BoroCT2020, Length: 2325, dtype: int64
